In [6]:
#@title Install Data Commons
!pip install --upgrade --quiet datacommons


In [7]:
#@title Install RDKit
# Install rdkit for molecular similairty calculation
# source: https://gist.github.com/philopon/a75a33919d9ae41dbed5bc6a39f5ede2

import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
    install()



add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
INFO:__main__:add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
python version: 3.10.12
INFO:__main__:python version: 3.10.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
INFO:__main__:fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
INFO:__main__:done
installing miniconda to /root/miniconda
INFO:__main__:installing miniconda to /root/miniconda
done
INFO:__main__:done
installing rdkit
INFO:__main__:installing rdkit


CalledProcessError: Command '['/root/miniconda/bin/conda', 'install', '--yes', '-c', 'rdkit', 'python==3.10.12', 'rdkit']' returned non-zero exit status 1.

In [ ]:
# Disease Selector Widget
import ipywidgets as widgets

# Data Commons Query
import datacommons as dc

# RDKit Analysis and Ranking
import pandas as pd
from collections import defaultdict
from ast import literal_eval
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

In [ ]:
#@title Set Disease Name
query_str = '''
SELECT ?name
WHERE {
?a typeOf Disease .
?a commonName ?name .
}
'''
result = dc.query(query_str)
disease_name_list = sorted([row['?name'] for row in result], key=str.lower)
disease_selector = widgets.Dropdown(options=disease_name_list, value="Crohn's disease", description='Disease:')

disease_selector

In [ ]:
disease_name = disease_selector.value
# This query finds a node in Data Commons that is of type Disease and has the
# common name of the disease we are looking for
query_str = '''
SELECT ?dcid
WHERE {{
?dcid typeOf Disease .
?dcid commonName "{0}" .
}}
'''.format(disease_name)

result = dc.query(query_str)
print('query result: ' + str(result))

disease_dcid = ''

# Extract the dcid of the disease and save it to disease_dcid
for row in result:
  disease_dcid = row['?dcid']

print(disease_dcid)

In [ ]:
# find all ChemicalCompoundDiseaeTreatment nodes that includes the disease
query_str = '''
SELECT ?cdt_dcid
WHERE {{
?disease dcid "{0}" .
?cdt_dcid typeOf 	ChemicalCompoundDiseaseTreatment .
?cdt_dcid diseaseID ?disease .
}}
'''.format(disease_dcid)

result = dc.query(query_str)
print('query result: ' + str(result))

treatment_compounds = []

if not result:
  print('no treatments found')

else:
  # extract just the dcids of the ChemicalCompoundDiseaeTreatment nodes
  cdt_dcids = set()
  for row in result:
    cdt_dcids.add(row['?cdt_dcid'])
  print('CDT dcids: ' + str(cdt_dcids))

  # Use Data Commons python API to get all of the compoundIDs from each CDT Node
  compounds = dc.get_property_values(cdt_dcids, 'compoundID')
  print('raw values:' + str(compounds))

  # Lets reformat this retreived data into a single list of compounds so it is
  # easier to iterate through in the future
  treatment_compounds = [drug for cdt, drug_list in compounds.items() for drug in drug_list ]
  print('treatment compounds: ' + str(treatment_compounds))
  print(len(treatment_compounds))

In [ ]:
# This query finds all ChemicalCompoundDiseaeContraindication nodes that the disease is connected to
query_str = '''
SELECT ?cdc_dcid
WHERE {{
?disease dcid "{0}" .
?cdc_dcid typeOf 	ChemicalCompoundDiseaseContraindication .
?cdc_dcid diseaseID ?disease .
}}
'''.format(disease_dcid)

result = dc.query(query_str)

inadvisable_compounds = []

if not result:
  print('no contraindications found')
else:
  contraindication_dcids = set()
  for row in result:
    contraindication_dcids.add(row['?cdc_dcid'])
  print(contraindication_dcids)

  compounds = dc.get_property_values(contraindication_dcids, 'compoundID')
  print('raw values: ' + str(compounds))

  # reformat compounds into iterable list
  inadvisable_compounds = [drug for cdc, drug_list in compounds.items() for drug in drug_list ]
  print(inadvisable_compounds)
  print(len(inadvisable_compounds))

no contraindications found


In [ ]:
# This query finds all DiseaseGeneAssociation nodes involving the disease
query_str = '''
SELECT ?dga_dcid
WHERE {{
?disease dcid "{0}" .
?dga_dcid typeOf 	DiseaseGeneAssociation .
?dga_dcid diseaseOntologyID ?disease .
}}
'''.format(disease_dcid)

all_associated_genes = []
hg38_associated_genes = []

result = dc.query(query_str)

if not result:
  print('no disease-gene associations found')
else:
  # retreive the gene dcids involved in all of the found DiseaseGeneAssociation
  # nodes, then limit to hg38
  dga_dcids = set()
  for row in result:
    dga_dcids.add(row['?dga_dcid'])
  print('disease-gene asscociations: ' + str(dga_dcids))

  genes_raw = dc.get_property_values(dga_dcids, 'geneID')
  print('raw values: ' + str(genes_raw))

  all_associated_genes = [gene for cdc, gene_list in genes_raw.items() for gene in gene_list ]
  print('all associated genes: ' + str(all_associated_genes))
  print(len(all_associated_genes))

  # limit to hg38 genome assembly
  hg38_associated_genes = [gene for gene in all_associated_genes if gene.startswith('bio/hg38')]
  print('associated hg38 genes: ' + str(hg38_associated_genes))
  print(len(hg38_associated_genes))

In [ ]:
# Find ChemicalCompoundGeneAssociation nodes involving the newly found
# associated genes
query_str = '''
SELECT ?cga_dcid ?gene
WHERE {{
?gene dcid ("{0}") .
?cga_dcid typeOf ChemicalCompoundGeneAssociation .
?cga_dcid geneID ?gene .
}}
'''

result = conduct_query(query_str, hg38_associated_genes)

candidate_to_genes = {}
print(result)

if not result:
  print('no drug-gene associations found')
else:
  # retreive the compound dcids involved in the found
  # ChemicalCompoundGeneAssociation nodes
  cga_list = [row['?cga_dcid'] for row in result]
  cga_to_candidate = dc.get_property_values(cga_list, 'compoundID')
  print(cga_to_candidate)

  for row in result:
    candidate = cga_to_candidate[row['?cga_dcid']][0]

    # ensure we have limited targets to hg38
    if not row['?gene'].startswith('bio/hg38'):
      continue
    # save candidate to gene targets relationship
    if candidate in candidate_to_genes:
      candidate_to_genes[candidate].add(row['?gene'])
    else:
      candidate_to_genes[candidate] = {row['?gene']}

print(candidate_to_genes)
print(len(candidate_to_genes))

In [ ]:
len(result)

In [ ]:
# Find ChemicalCompoundGeneAssociation nodes involving the treatment compounds
query_str = '''
SELECT ?cga_dcid ?reference_drug
WHERE {{
?reference_drug dcid ("{0}") .
?cga_dcid typeOf ChemicalCompoundGeneAssociation .
?cga_dcid compoundID ?reference_drug
}}
'''
rows = conduct_query(query_str, treatment_compounds)

gene_to_references = {}
print(rows)

if not rows:
  print('no reference compound-gene associations found')
else:
  # extract gene target to treatment compound information
  for row in rows:
    gene = dc.get_property_values([row['?cga_dcid']], 'geneID')[row['?cga_dcid']][0]

    # limit to using genome assembly hg38
    if not gene.startswith('bio/hg38'):
      continue
    # save gene target to treatment compounds mapping
    if gene in gene_to_references:
      gene_to_references[gene].add(row['?reference_drug'])
    else:
      gene_to_references[gene] = {row['?reference_drug']}
print(gene_to_references)

In [ ]:
candidates = list(candidate_to_genes.keys()) # all candidates
candidate_to_established_target = defaultdict(list) # group of candidates to be ranked
candidate_to_novel_target = defaultdict(list) # group of candidates to remain unranked

# determine which group each candidate falls under
for candidate in candidates:
  if (candidate in treatment_compounds) or (candidate in inadvisable_compounds):
    continue
  gene_targets = list(candidate_to_genes[candidate])
  for gene_target in gene_targets:
    # check if gene is targeted by a treatment compound
    if gene_target in gene_to_references:
      candidate_to_established_target[candidate].append(gene_target)
    # otherwise add to list of candidates that shall remain unranked
    else:
      candidate_to_novel_target[candidate].append(gene_target)

# candidates that share targets with reference compounds:
print(candidate_to_established_target)
print(len(candidate_to_established_target))

# candidates with novel gene targets:
print(candidate_to_novel_target)
print(len(candidate_to_novel_target))
print('\n')
# create dataframe to store and print the unranked candidates
print('Unranked Candidates with Novel Gene Targets:')
unranked_candidate_df = pd.DataFrame(
    [(key, L) for (key, L) in candidate_to_novel_target.items()],
                columns=['candidate_dcid', 'gene_target_list'])
unranked_candidate_df

In [ ]:
# get SMILES for reference drugs, store in reference_drugs dict
query_str = '''
SELECT ?smiles ?drug
WHERE {{
?drug dcid ("{0}") .
?drug simplifiedMolecularInputLineEntrySystem ?smiles .
}}
'''

result = conduct_query(query_str, treatment_compounds)
print(result)

reference_drugs = {}

if not result:
  print('no smiles found')
else:
  for row in result:
    reference_drugs[row['?drug']] = row['?smiles']


In [ ]:
# get the candidates that have established gene targets
filtered_candidates = list(candidate_to_established_target.keys())
print(len(filtered_candidates))

# get SMILES of candidates, store in pandas dataframe candidate_df
query_str = '''
SELECT ?smiles ?drug
WHERE {{
?drug dcid ("{0}") .
?drug simplifiedMolecularInputLineEntrySystem ?smiles .
}}
'''

result = conduct_query(query_str, filtered_candidates)
ranked_candidate_df = pd.DataFrame(columns = ['candidate_dcid', 'smiles'])

if not result:
  print('no smiles found')
else:
  for row in result:
    ranked_candidate_df = ranked_candidate_df.append({'candidate_dcid': row['?drug'], 'smiles':row['?smiles']}, ignore_index=True)

# add the gene targets for each candidate in the dataframe
ranked_candidate_df['genes_list'] = ranked_candidate_df.candidate_dcid.apply(lambda dcid: candidate_to_established_target[dcid])
# print out resulting data frame
ranked_candidate_df

In [ ]:
def calculate_tanimoto_coeff(cand_smiles, ref_smiles):
  """Calculates Tanimoto Coefficient given SMILES of two compounds.

  Find the structural similarity of candidate compound and reference compound
  which share at least one gene target using Morgan Fingerprints and functions
  from RDKit.

  Args:
      cand_smiles: SMILES of a candidate compound.
      ref_smiles: SMILES of a reference compound.

  Returns:
      The calculated Tanimoto Coefficient for the two compounds.
  """
  # construct the molecules from the SMILES using RDKit function
  cand_mol = Chem.MolFromSmiles(cand_smiles)
  ref_mol = Chem.MolFromSmiles(ref_smiles)

  # create Morgan Fingerprints of the constructed molecules using RDKit function
  cand_fgprint = AllChem.GetMorganFingerprintAsBitVect(cand_mol,2)
  ref_fgprint = AllChem.GetMorganFingerprintAsBitVect(ref_mol,2)

  # calculate Tanimoto similarity coefficient from fingerprints using RDKit function
  similarity = DataStructs.TanimotoSimilarity(cand_fgprint,ref_fgprint)
  return similarity

In [ ]:
# given a single gene target and a candidate, find the highest tanimoto
# coefficient the candidate has with a reference compound that targets the same
# gene
def find_max_sim(gene, cand_smiles):
  """Finds the treatment compound most similar to the candidate compound out of
  all treatment compounds which target the given gene.

  Given a gene and SMILES of a candidate compound, determines the
  Tanimoto Coefficient between each treatment compound that targets the given
  gene and the given candidate SMILES. Uses the gene_to_references dictionary
  created in the Drug-Gene Associations Query section to map genes to the
  treatment compounds which target them. Uses the reference_drugs dictionary
  created in the Get Reference SMILES section to map treatment compound dcids
  to their SMILES.

  Args:
      gene: Dcid of the gene to use as the given gene target.
      cand_smiles: SMILES of a candidate compound.

  Returns:
      The highest determined Tanimoto Coefficient and the corresponding
      treatment compound in the form of a dictionary.
  """
  max = {'coeff': -1, 'ref':''}
  if gene in gene_to_references:
    for ref_mol in gene_to_references[gene]:
      ref_smiles = reference_drugs[ref_mol];
      sim = calculate_tanimoto_coeff(cand_smiles, ref_smiles)
      if sim > max['coeff']:
        max['coeff'] = sim
        max['ref'] = ref_mol

  return max

# create list of highest Tanimoto Coefficients for each gene target of a
# candidate
def find_gene_sims(row):
  """Creates a list of Tanimoto Coefficient for each gene that a candidate
  compound is known to target.

  Given a row from ranked_candidate_df, finds the maximum Tanimoto Coefficient
  for each gene target of the candidate. Saves the list of coefficients,
  corresponding treatment compound dcids, and the pairings of coeffcients to
  treatment compounds as separate columns in ranked_candidate_df.

  Args:
      row: Row of ranked_candidate_df containing columns for the candidate dcid,
      candidate SMILES, and the candidates' gene targets.

  Returns:
      The given row with the new columns 'tanimoto_coefficients',
      'reference_compounds', and 'similarity_pair' appended.
  """
  row['tanimoto_coefficients'] = []
  row['reference_compounds'] = []
  row['similarity_pair'] = []

  sim_pairs = []
  for gene in row['genes_list']:
    sim_pairs.append(find_max_sim(gene, row['smiles']))
  sorted_sims = sorted(sim_pairs, key = lambda i: i['coeff'], reverse=True)
  for pair in sorted_sims:
    row['tanimoto_coefficients'].append(pair['coeff'])
    row['reference_compounds'].append(pair['ref'])
    row['similarity_pair'].append(pair)
  return row

# find the list of tanimoto coefficients
ranked_candidate_df = ranked_candidate_df.apply(lambda row: find_gene_sims(row), axis=1)
# sort the data frame
ranked_candidate_df = ranked_candidate_df.sort_values(by='tanimoto_coefficients', ascending=False, ignore_index=True)
# drop duplicates caused by compounds with multiple SMILES
ranked_candidate_df.tanimoto_coefficients = ranked_candidate_df.tanimoto_coefficients.astype(str)
ranked_candidate_df = ranked_candidate_df.drop_duplicates(subset=['candidate_dcid', 'tanimoto_coefficients'],ignore_index=True)
ranked_candidate_df.tanimoto_coefficients = ranked_candidate_df.tanimoto_coefficients.apply(literal_eval)
# print top 10 compounds
ranked_candidate_df.head(10)

In [ ]:
# View Top 10 Structures
best_mols = ranked_candidate_df.smiles[:10].apply(Chem.MolFromSmiles)
Chem.Draw.MolsToGridImage(best_mols,molsPerRow=5)

In [ ]:
# View top molecule vs most similar treatment with shared gene target
top_mol = Chem.MolFromSmiles(ranked_candidate_df.smiles[0])
ref_mol = Chem.MolFromSmiles(reference_drugs[ranked_candidate_df.reference_compounds[0][0]])
Chem.Draw.MolsToGridImage([top_mol, ref_mol], molsPerRow=2)

In [ ]:
top_mol_dcid = ranked_candidate_df['candidate_dcid'].tolist()[0]
print('top candidate: ' + top_mol_dcid)

# get outgoing triple labels for the drug
out_labels = dc.get_property_labels([top_mol_dcid], out=True)[top_mol_dcid]
print('\nDrug properties we could examine:')
for label in out_labels:
  print(label)

print('\nSome specific values for bio/CHEMBL532:')
for label in ['administrationRoute', 'commonName', 'dosageForm', 'drugName', 'maximumFDAClinicalTrialPhase', 'propietaryName', 'recognizingAuthority', 'url']:
  value = dc.get_property_values([top_mol_dcid], label, out=True)[top_mol_dcid]
  print(label + ': ' + str(value))

In [ ]:
# look at specific drug strength information
strength_dcid = ''
try:
  strength_dcid = dc.get_property_values([top_mol_dcid], 'availableStrength', out=True)[top_mol_dcid]

  out_labels = dc.get_property_labels(strength_dcid, out=True)[strength_dcid[0]]
  print(strength_dcid)

  print('---Outgoing Drug Strength Triples---')
  for label in out_labels:
    value = dc.get_property_values(strength_dcid, label, out=True)[strength_dcid[0]]
    print(label + ': ' + str(value))
except:
    print('There is no FDA data for this compound.')

if strength_dcid:
  print('\n')

  # look at FDA Application
  fda_app_dcid = dc.get_property_values(strength_dcid, 'submittedFDAApplication', out=True)[strength_dcid[0]]

  print(fda_app_dcid)

  out_labels = dc.get_property_labels(fda_app_dcid, out=True)[fda_app_dcid[0]]
  print('---Outgoing FDA Application Triples---')
  for label in out_labels:
    value = dc.get_property_values(fda_app_dcid, label, out=True)[fda_app_dcid[0]]
    print(label + ': ' + str(value))

In [ ]:
# get incoming triple labels for the drug
in_labels = dc.get_property_labels([top_mol_dcid], out=False)[top_mol_dcid]
print('in labels: ' + str(in_labels))
print('\n---Incoming Triples---')
for label in in_labels:
  value = dc.get_property_values([top_mol_dcid], label, out=False)[top_mol_dcid]
  print(label + ' of ' + str(value))

try:
  print('\n' + top_mol_dcid + ' is known to treat:')
  # we want to find the side effects, we can do so by specifiying the type of
  # incoming value we want to find
  treatment_nodes = dc.get_property_values([top_mol_dcid], 'compoundID', out=False, value_type='ChemicalCompoundDiseaseTreatment')[top_mol_dcid]
  treatment_lists = list(dc.get_property_values(treatment_nodes, 'diseaseID', out=True).values())
  diseases = [disease_list[0] for disease_list in treatment_lists]

  disease_names = list(dc.get_property_values(diseases, 'commonName', out=True).values())
  for name in disease_names:
    print(name)
except:
  print('There are no ChemicalCompoundDiseaseTreatment nodes involving this compound.')